# CS202: Compiler Construction

## In-class Exercises, Week of 02/08/2021

----
# PART I: Rvar language & interpreter

The abstract syntax definition for RVar is below.

In [1]:
from dataclasses import dataclass
from typing import List, Set, Dict, Tuple
from cs202_support.base_ast import AST, print_ast

@dataclass
class RVarExp(AST):
    pass

@dataclass
class Int(RVarExp):
    val: int

@dataclass
class Var(RVarExp):
    var: str

@dataclass
class Let(RVarExp):
    x: str
    e1: RVarExp
    body: RVarExp

@dataclass
class Prim(RVarExp):
    op: str
    args: List[RVarExp]


## Question 1

Write an abstract syntax tree for the program: `let x = 5 in x + 3`.

In [2]:
ast = Let('x', Int(5), Prim('+', [Var('x'), Int(3)]))
print(print_ast(ast))

Let(
 x,
 Int(5),
 Prim(
  +,
  [
   Var(x),
   Int(3)
  ]))


In [3]:
ast = Let('x', Let('y', Int(3), Prim('+', [Var('y'), Int(2)])),
          Prim('+', [Var('x'), Int(5)]))
print(print_ast(ast))

Let(
 x,
 Let(
  y,
  Int(3),
  Prim(
   +,
   [
    Var(y),
    Int(2)
   ])),
 Prim(
  +,
  [
   Var(x),
   Int(5)
  ]))


## Question 2

What is the value of the program in question 1?

`let x = 5 in x + 3` is the same as `replace x with 5 in "x+3" = 5 + 3 = 8`

## Question 3

Write an interpreter for Rvar.

In [4]:
def eval_rvar(e: RVarExp, env: Dict[str, int]) -> int:
    if isinstance(e, Int):
        return e.val
    elif isinstance(e, Var):
        # look up the variable in the environment
        var_name = e.var
        var_val = env[var_name]
        return var_val
        pass
    elif isinstance(e, Let):
        let_var = e.x
        let_rhs : RVarExp = e.e1
        let_body = e.body
        
        let_rhs_val = eval_rvar(let_rhs, env)
        
        #env[let_var] = let_rhs_val # updates the env with a side effect
        new_env = {**env, let_var : let_rhs_val} # updates the env without a side effect
        
        let_body_val = eval_rvar(let_body, new_env)
        return let_body_val
        
    elif isinstance(e, Prim):
        if e.op == '+':
            e1, e2 = e.args
            v1 = eval_rvar(e1, env)
            v2 = eval_rvar(e2, env)
            return v1 + v2
    else:
        raise Exception('unknown expression', e)

In [5]:
test_ast = Let('x', Int(5),
              Prim('+', [Var('x'), Int(6)]))
eval_rvar(test_ast, {})

11

In [6]:
# TEST CASE
eval_rvar(ast, {})

10

## Question 4

What is the value of the following program?

In [7]:
ast = Let('x', Int(5), Let('x', Int(6), Var('x')))
print(print_ast(ast))
eval_rvar(ast, {})

Let(
 x,
 Int(5),
 Let(
  x,
  Int(6),
  Var(x)))


6

The value is 6 because the second binding for x shadows the first binding

----
# PART II: uniquify

## Question 5

Why do we need this pass?

- It makes bugs in later passes less likely because variable names are unique
- It allows re-ordering bindings without changing the program's meaning

## Question 6

Write a new version of the program from above such that all the variable names are unique.

In [8]:
ast = Let('x', Int(5), Let('x', Int(6), Var('x')))
print(print_ast(ast))

Let(
 x,
 Int(5),
 Let(
  x,
  Int(6),
  Var(x)))


In [9]:
unique_ast = Let('x1', Int(5), Let('x2', Int(6), Var('x2')))
print(print_ast(unique_ast))

Let(
 x1,
 Int(5),
 Let(
  x2,
  Int(6),
  Var(x2)))


## Question 7

Describe a recursive function for making variable names unique.

The function follows the same structure as the interpreter. 
In addition to a RVar expression, it takes an environment. 
The environment maps variable names to their new (unique) names. 

- Case int: return the integer
- Case var: look up the var's new name in the env and return it
- Case Let: recursively call uniquify on e1 (the rhs of the binding); generate a new name "xN" where N is a unique number; extend environment by binding x to xN, recursively call uniquify on the body of the Let 
- Case Plus: recursively call uniquify on sub-expressions

## Question 8

What does the environment look like when `uniquify` gets to the expression `Prim('+', [Var('x'), Int(6)])`? What about when it gets to the expression `Var('x')`?

In [10]:
ast = Let('x', Int(5), Prim('+', [Var('x'), Int(6)]))
# let x = 5 in x + 6

In [ ]:
first_env = {'x': 'x1'}
second_env = {'x': 'x1'}

----
# PART III: remove-complex-opera*

## Question 8

Consider this translation of an expression to assembly language. What is wrong with it?

In [13]:
ast = Prim('+', [Int(1), Prim('+', [Int(2), Int(3)])])
print(print_ast(ast))

asm = """
movq $2, %rax
addq $1, (addq $3, %rax)
"""

Prim(
 +,
 [
  Int(1),
  Prim(
   +,
   [
    Int(2),
    Int(3)
   ])
 ])


It has nested arguments to assembly language instructions. Assembly doesn't allow this.

## Question 9

Which AST nodes in the language RVar are **atomic**?

Integers and variables

## Question 10

Why do we need this pass? What is the form of its output?

- Assembly language doesn't allow nested expressions
- RVar has nested expressions
- We need to un-nest expressions

## Question 11

Convert the program from earlier into A-normal form.

In [14]:
ast = Prim('+', [Int(1), Prim('+', [Int(2), Int(3)])])
print(print_ast(ast))

Prim(
 +,
 [
  Int(1),
  Prim(
   +,
   [
    Int(2),
    Int(3)
   ])
 ])


In [13]:
anf_ast = Let('tmp', Prim('+', [Int(2), Int(3)]), 
            Prim('+', [Int(1), Var('tmp')]))
print(print_ast(anf_ast))

Let(
 tmp,
 Prim(
  +,
  [
   Int(2),
   Int(3)
  ]),
 Prim(
  +,
  [
   Int(1),
   Var(tmp)
  ]))


## Question 12

Describe a recursive procedure to perform the *remove-complex-opera* pass.

- rco-exp operates on expressions
    - when you get to a + expression, call rco-atm on the arguments
- rco-atm operates on things that need to be atomic